If we consider the slightly more complex system:

A + cat <---> B -> C + cat

where labeled reaction A-d10 is added with a time delay.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

reactions = [
    ('k1', ['A', 'cat'], ['B'],),
    ('k-1', ['B'], ['A', 'cat'],),
    ('k2', ['B'], ['C', 'cat']),

    # labeled
    ('k1', ['A-d10', 'cat'], ['B-d10'],),
    ('k-1', ['B-d10'], ['A-d10', 'cat'],),
    ('k2', ['B-d10'], ['C-d10', 'cat'])
]

rate_constants_real = {'k1': 0.3, 'k-1': 0.05, 'k2': 0.5}
concentration_initial = {'A': 1, 'cat': 1 / 5}
concentration_labeled = {'A-d10': 0.5}
dilution_factor = 1 / 2
time_pre = np.linspace(0, 10, 50)  # low resolution
time_post = np.linspace(10, 90, 8 * 50)

In [ ]:
from delayed_reactant_labeling.predict_new import DRL

drl_real = DRL(rate_constants=rate_constants_real, reactions=reactions)
real_data_pre, real_data = drl_real.predict_concentration(
    t_eval_pre=time_pre,
    t_eval_post=time_post,
    dilution_factor=dilution_factor,
    initial_concentrations=concentration_initial,
    labeled_concentration=concentration_labeled)

fig, axs = plt.subplots(1, 2, sharey='row', figsize=(10, 4), layout='tight', width_ratios=(1, 5))
real_data_pre.to_pandas().plot('time', ax=axs[0])
real_data.to_pandas().plot('time', ax=axs[1])

In [ ]:
import polars as pl

rng = np.random.default_rng(42)

fig, ax = plt.subplots()
fake_data = []
for col in real_data.columns[:-1]:  # last column contains time array
    noise = rng.normal(loc=0, scale=0.10 * real_data[col].mean(), size=(real_data.shape[0]))
    fake_col = real_data[col] + noise  # noise is loosely based on intensity
    fake_col[fake_col < 0] = 0  # no negative intensity
    fake_data.append(fake_col)
    ax.scatter(real_data['time'], fake_col, label=col, marker='.')

ax.legend(ncol=4)
fake_data.append(real_data['time'])
fake_data = pl.DataFrame(fake_data, real_data.columns)

In [ ]:
import pandas as pd
from delayed_reactant_labeling.optimize import RateConstantOptimizerTemplate

class RateConstantOptimizer(RateConstantOptimizerTemplate):
    @staticmethod
    def create_prediction(x: np.ndarray, x_description: list[str]) -> pl.DataFrame:
        # this is also the location where the rate constants can be manipulated easily
        # e.g. rate_constants['k1'] = 0.1, would fixate this value

        rate_constants = pd.Series(x, x_description)
        drl = DRL(reactions=reactions, rate_constants=rate_constants)
        _, pred_labeled = drl.predict_concentration(
            t_eval_pre=time_pre,
            t_eval_post=time_post,
            initial_concentrations=concentration_initial,
            labeled_concentration=concentration_labeled,
            dilution_factor=dilution_factor,
            rtol=1e-8,
            atol=1e-8, )

        # the output can also be manipulated before analyzing it further. Allows grouping of certain groups,
        # when we know that they are measured as the same compound in the mass spectrometer.
        return pred_labeled

    @staticmethod
    def calculate_curves(data: pl.DataFrame) -> dict[str, pl.Series]:
        curves = {}
        for chemical in ['A', 'B', 'C']:
            chemical_sum = data[[chemical, f'{chemical}-d10']].sum(axis=1)
            curves[f'ratio_{chemical}'] = data[chemical] / chemical_sum
            # curves[f'TIC_{chemical}'] = data[chemical] / chemical_sum[-50:].mean()
            # curves[f'TIC_{chemical}-d10'] = data[f'{chemical}-d10'] / chemical_sum[-50:].mean()
        return curves

In [ ]:
def METRIC(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return np.average(np.abs(y_pred - y_true), axis=0)

RCO = RateConstantOptimizer(raw_weights={}, experimental=fake_data, metric=METRIC)
dimension_description = ['k1', 'k-1', 'k2']
bounds = [(1e-6, 100),    # k1
          (0, 50),        # k-1
          (1e-6, 100),]   # k2
x0 = [5, 5, 5]

path = 'examples_optimization/example_optimize_simple_singular/'
RCO.optimize(
    path=path,
    x0=x0,
    bounds=bounds,
    x_description=dimension_description,
    maxiter=10000,
    _overwrite_log=True
)

In [ ]:
progress = RCO.load_optimization_progress(path)
pd.DataFrame([progress.best_X.values, np.array(list(rate_constants_real.values()))], index=['found', 'real'], columns=progress.best_X.index)

In [ ]:
fig, axs = plt.subplots(3, 1, tight_layout=True, figsize=(8, 8), squeeze=False)

true_curves = RCO.experimental_curves
best_pred = RCO.create_prediction(x=progress.best_X, x_description=progress.x_description)
errors = RCO.calculate_error_functions(best_pred)
errors = RCO.weigh_errors(errors)
pred_curves = RCO.calculate_curves(best_pred)

for i, chemical in enumerate(['A', 'B', 'C']):
        # plot label ratio
        axs[i, 0].plot(time_post, pred_curves[f"ratio_{chemical}"], color=f"C{i}",
                       label=f"{chemical} MAE: {errors[f'ratio_{chemical}']:.3f}")
        axs[i, 0].scatter(time_post, true_curves[f"ratio_{chemical}"],
                          color=f"C{i}", alpha=0.4, marker=".", s=1)

        # the curve of the labeled compound is the same, by definition, as 1 - unlabeled
        axs[i, 0].plot(time_post, 1 - pred_curves[f"ratio_{chemical}"], color="tab:gray")
        axs[i, 0].scatter(time_post, 1 - true_curves[f"ratio_{chemical}"], color="tab:gray", alpha=0.4, marker=".", s=1)
        axs[i, 0].legend()

In [ ]:
from tqdm import tqdm
fig, ax = plt.subplots()
k1 = np.linspace(0, 10, 50)
kr1 = np.linspace(0, 10, 50)
k2 = 0.5

im = np.zeros((k1.shape[0], kr1.shape[0]))
for i in tqdm(range(k1.shape[0])):
    for j in range(kr1.shape[0]):
        pred = RCO.create_prediction(x=np.array([k1[i], kr1[j], k2]), x_description=['k1', 'k-1', 'k2'])
        error = RCO.calculate_total_error(RCO.calculate_error_functions(pred))
        im[i, j] = error

In [ ]:
im_reduced = np.log10(im)
im_reduced_min = ic(np.nanmin(im_reduced))
im_reduced[im_reduced > im_reduced_min + 100] = im_reduced_min + 100

fig, ax = plt.subplots()
image = ax.imshow(im_reduced, origin="lower", aspect="auto")
ax.set_xlabel('k1')
ax.set_ylabel('k-1')
ax.set_title("k2 = 0.5")

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", "5%", pad="3%")
fig.colorbar(image, cax=cax, label="MAE")

In [ ]:
path_multiple = 'examples_optimization/example_optimize_simple_multiple/'
RCO.optimize_multiple(path=path_multiple, n_runs=50, bounds=bounds, x_description=dimension_description, n_jobs=-2)

In [ ]:
from delayed_reactant_labeling.visualize import VisualizeMultipleSolutions
VMS = VisualizeMultipleSolutions('examples_optimization/example_optimize_simple_multiple/optimization_multiple_guess/', 50)